# Too slow if read part file, waiting for better solution.

In [6]:
import os
import SimpleITK as sitk
import numpy as np
import time

In [7]:
start_t = time.time()
count = 0
for i in range(1,11):
    count+=1
    patch_ct_path = r"test_data\image{}.mha".format(i)
    patch_image=sitk.ReadImage(patch_ct_path)
    patch_image_arry = sitk.GetArrayFromImage(patch_image)
end16_t = time.time()
print(end16_t-start_t)
print(count)

0.06682014465332031
10


In [8]:
patch_image_arry.shape

(128, 128, 32)

In [9]:
import struct

def read_patch(patch_size,start_point,file_shape,file):
    pic = []
    utime_read = 0
    utime_convert = 0
    with open(file,"rb") as f:
        for xi in range(start_point[0],start_point[0]+patch_size[0]):
            for yi in range(start_point[1],start_point[1]+patch_size[1]):
                start = xi*file_shape[1]*file_shape[2]+yi*file_shape[2]+start_point[2]
                time1 = time.time()
                f.seek(start)
                nums = f.read(patch_size[2]*2)
                time2 = time.time()
                #for i in range(0,patch_size[2]*2,2):
                nums = list(struct.unpack("<h$", nums))
                    #num = int.from_bytes(nums[i:i+2],byteorder='little', signed=True)
                pic+=nums
                time3 = time.time()
                utime_read+=time2-time1
                utime_convert += time3-time2

    myresult = np.array(pic).reshape(patch_size)

    return myresult, utime_read, utime_convert


In [10]:
patch_size = (128,128,32)
start_point = (128,128,32)
file_shape = (512,512,147)
time_all = 0
start_t = time.time()
count = 0
utime_read_all = 0
utime_convert_all = 0
for i in range(0,100,10):
    start_point = (i,i,32)
    patch_image_arry, utime_read, utime_convert = \
        read_patch(patch_size,start_point,file_shape,"test_data\speed_test_int.txt")
    count+=1
    utime_read_all+=utime_read
    utime_convert_all+=utime_convert
end16_t = time.time()
print(end16_t-start_t)
print(count)
print(utime_read_all, utime_convert_all)

error: bad char in struct format

In [ ]:
example_ct_path = r"test_data\c_ct.mha"
image=sitk.ReadImage(example_ct_path)
image_arry = sitk.GetArrayFromImage(image)

In [ ]:
# x,y,z
list_ct = list(image_arry.flatten())
for i in range(len(list_ct)):
    list_ct[i] = str(list_ct[i])

In [ ]:
list_ct_int = list(image_arry.flatten())

In [ ]:
with open("test_data\speed_test_int.txt","wb") as f:
    f.writelines(list_ct_int)

In [ ]:
with open("speed_test.txt","w") as f:
    f.writelines([line+'\n' for line in list_ct])

In [ ]:
print(list_ct_int[0].dtype)
print(len(list_ct_int))

In [ ]:
with open("speed_test.txt","r") as fin:
    #fin.seek(0)
    data = fin.readlines()
len(data)

In [ ]:
data

In [ ]:
pic = []

for i in range(0,1000,2):
    num = int.from_bytes(data[i:i+2],byteorder='little', signed=True)
    pic.append(num)

In [ ]:
result = read_patch(patch_size,start_point,file_shape,"speed_test_int.txt")